### Demonstrate the fitting cycle
Best practice found:
 - learn_one_cycle(10)
 - fine_tune(5), fine_tune(5), fine_tune(5)
 - 4th round of fine_tune(5) overfits

In [ ]:
from fastai2.vision.all import *
import sys
sys.path.append('..')

In [2]:
%load_ext autoreload
%autoreload 2

from modules.trainutils import build_dl, piece_class_parse
from modules.trainutils import my_metrics, stratify_sample
from modules.trainutils import show_cf

In [3]:
test_path = Path('../../../other-chess-data/regulation-test-1')
train_path = Path('../../../rf-chess-data/cropped_v1/')

In [4]:
_mult =         1.0
_max_lighting = 0.9  # 0.95
_max_warp    =  0.4  # 0.3
_max_rotate  =  20.  # 30.

_resize_method = ResizeMethod.Crop #ResizeMethod.Pad, ResizeMethod.Squish
_pad_mod = 'zeros'  #'reflection'

Crop = Resize(128, ResizeMethod.Pad, pad_mode='zeros')

Augs = aug_transforms(mult=_mult, 
                      max_lighting=_max_lighting,
                      p_lighting=0.75, 
                      max_warp=_max_warp,
                      max_rotate=_max_rotate,
                     )

train_dl = ImageDataLoaders.from_name_func(
                train_path, 
                get_image_files(train_path),
                # stratify_sample(train_path, n=n, np_seed=seed),
                valid_pct=0.2, 
                seed=None,
                label_func=piece_class_parse, 
                item_tfms=Crop,
                batch_tfms=Augs,
                bs=32,
)

##### Initial fit_one_cyle(10) | loss 1.06

In [6]:
learn = cnn_learner(train_dl, resnet50, metrics=error_rate)
    
#     learn.cbs.pop(2)
#     learn.cbs.pop(1)

learn.fit_one_cycle(10)

test_dl = learn.dls.test_dl(get_image_files(test_path), 
                                with_labels=True)

loss, acc = my_metrics(learn, test_dl)

loss, acc

epoch,train_loss,valid_loss,error_rate,time
0,2.253057,0.239722,0.057491,00:12
1,1.214846,0.212660,0.055749,00:09
2,0.788840,0.094326,0.017422,00:09
3,0.525421,0.102062,0.015679,00:09
4,0.407647,0.082511,0.008711,00:09
5,0.345481,0.088711,0.015679,00:09
6,0.303169,0.086281,0.010453,00:09
7,0.255237,0.078481,0.008711,00:09
8,0.238632,0.078124,0.008711,00:09
9,0.253925,0.076989,0.008711,00:09


(1.0603284080823263, 0.7166666666666667)

##### tune #1: fine_tune(5)  | loss: 1.06 -> .87

In [7]:
learn.fine_tune(5)

epoch,train_loss,valid_loss,error_rate,time
0,0.305900,0.127887,0.024390,00:09


epoch,train_loss,valid_loss,error_rate,time
0,0.323189,0.104149,0.010453,00:13
1,0.347716,0.104802,0.010453,00:12
2,0.286411,0.129561,0.006969,00:12
3,0.211498,0.108686,0.006969,00:13
4,0.186563,0.114341,0.006969,00:13


In [8]:
test_dl = learn.dls.test_dl(get_image_files(test_path), 
                                with_labels=True)

loss, acc = my_metrics(learn, test_dl)

loss, acc

(0.878013531366984, 0.7833333333333333)

##### tune #2: fine_tune(5)  | loss: .87 -> .77

In [9]:
learn.fine_tune(5)

test_dl = learn.dls.test_dl(get_image_files(test_path), 
                                with_labels=True)

loss, acc = my_metrics(learn, test_dl)

loss, acc

epoch,train_loss,valid_loss,error_rate,time
0,0.177497,0.122240,0.006969,00:10


epoch,train_loss,valid_loss,error_rate,time
0,0.194184,0.110719,0.006969,00:12
1,0.232876,0.088339,0.006969,00:13
2,0.209727,0.102828,0.006969,00:13
3,0.178355,0.101534,0.006969,00:13
4,0.147595,0.096223,0.006969,00:13


(0.7688380519549052, 0.8166666666666667)

In [11]:
def my_export(learn, model_fn='tmp-model.pkl'):
    ' to verify run: !ls ../models -sh '
    old_path = learn.path
    learn.path = Path('../models')
    learn.export(model_fn)
    learn.path = old_path

In [12]:
my_export(learn)

In [14]:
!ls ../models -1sh

total 787M
 52M bing-clean-1.pkl
 52M bing-clean-2.pkl
 48M chess1.pkl
 57M demo-model-1.pkl
135M demo-model-1.pth
188M resnet101-acc73.pkl
 99M resnet50-acc75.pkl
 57M standard-piece-2.pkl
104M tmp-model.pkl


##### tune #3: fine_tune(5)  test_loss: .77 -> .62

In [15]:
learn.fine_tune(5)

test_dl = learn.dls.test_dl(get_image_files(test_path), 
                                with_labels=True)

loss, acc = my_metrics(learn, test_dl)

loss, acc

epoch,train_loss,valid_loss,error_rate,time
0,0.152185,0.098396,0.006969,00:10


epoch,train_loss,valid_loss,error_rate,time
0,0.156135,0.104618,0.006969,00:13
1,0.215758,0.120360,0.008711,00:13
2,0.189185,0.087822,0.006969,00:12
3,0.140009,0.083669,0.006969,00:13
4,0.127673,0.086878,0.006969,00:13


(0.6273680170377095, 0.8333333333333334)

In [16]:
my_export(learn, model_fn='tmp-2.pkl')

##### tune #4: Too much! loss: .62 ->  .85

In [17]:
learn.fine_tune(5)

test_dl = learn.dls.test_dl(get_image_files(test_path), 
                                with_labels=True)

loss, acc = my_metrics(learn, test_dl)

loss, acc

epoch,train_loss,valid_loss,error_rate,time
0,0.132852,0.099886,0.006969,00:11


epoch,train_loss,valid_loss,error_rate,time
0,0.140575,0.097690,0.008711,00:13
1,0.166928,0.092931,0.006969,00:13
2,0.154774,0.099097,0.010453,00:13
3,0.109076,0.082225,0.006969,00:13
4,0.108334,0.079644,0.006969,00:13


(0.8565141757329305, 0.7)

##### Try one more time to regain perf - nope! (loss goes slighly up again)

In [18]:
learn.fine_tune(10)

test_dl = learn.dls.test_dl(get_image_files(test_path), 
                                with_labels=True)

loss, acc = my_metrics(learn, test_dl)

loss, acc

epoch,train_loss,valid_loss,error_rate,time
0,0.099625,0.088658,0.006969,00:10


epoch,train_loss,valid_loss,error_rate,time
0,0.117652,0.081735,0.006969,00:13
1,0.116166,0.082764,0.006969,00:13
2,0.133151,0.104370,0.008711,00:13
3,0.160192,0.096288,0.006969,00:13
4,0.140264,0.076848,0.008711,00:13
5,0.121205,0.085197,0.006969,00:13
6,0.096111,0.088716,0.006969,00:13
7,0.104019,0.083238,0.006969,00:13
8,0.084904,0.080907,0.006969,00:13
9,0.088224,0.082097,0.006969,00:13


(0.8944990356763204, 0.75)